In [2]:
import sys
from pathlib import Path

# Add project root to Python path
# This assumes your notebook is in the notebooks/ directory
project_root = Path.cwd().parent  # goes up one level from notebooks/ to project root
sys.path.append(str(project_root))

from src.data.download.rest_downloader import (
    get_arcgis_services_to_pd,
    download_and_create_cog,
)

# Orthophotos

In [ ]:
service_url = "https://gis.stmk.gv.at/image/rest/services/OGD_DOP"

# Test getting services
print("Fetching available services...")
services_df = get_arcgis_services_to_pd(service_url)
display(services_df.head(50))

In [8]:
# Define which services to use
services = [
    "Falschfarben_2008_2011",
    "Falschfarben_2013_2015",
    "Falschfarben_2016_2018",
    "Falschfarben_2019_2021",
    "Falschfarben_2022_2024",
    "Flug_2003_2007_RGB",
    "Flug_2008_2011_RGB",
    "Flug_2013_2015_RGB",
    "Flug_2016_2018_RGB",
    "Flug_2019_2021_RGB",
    "Flug_2022_2024_RGB",
    "SW_1994_2001",
]

services_to_download_df = services_df.loc[services_df["service_name"].isin(services)]

In [ ]:
# loop through services_df and download all tiles from each service using the service_url and service_name
for index, row in services_to_download_df.iterrows():
    service_name = row["service_name"]
    print(f"Downloading tiles from service: {service_name}")

    download_and_create_cog(
        service_url=service_url,
        service_name=service_name,
        category="orthophoto",
        output_path=f"../data/processed/orthophotos_gis_stmk/{service_name.lower()}.tif",
        bbox_gpkg_path="../data/roi/Verschneidung_NPG_Johnsbachtal_Europaschutzgebiet_BBOX.gpkg",
        raw_data_folder=f"../data/raw/orthophoto_tiles_gis_stmk/{service_name.lower()}",
        out_nodata_value=0
    )

# Elevation Data

In [ ]:
service_url = "https://gis.stmk.gv.at/image/rest/services/OGD_Hoehen"
output_folder = Path("../data/processed/elevation_gis_stmk_photogrammetry_2022_2023")

# Test getting services
print("Fetching available services...")
services_df = get_arcgis_services_to_pd(service_url)
display(services_df.head(50))

In [ ]:
# Define which services to use
services = [
    "Luftbild_Hoehen_2022_2024_DOM_BEV_UTM33N_32633",
]

services_to_download_df = services_df.loc[services_df["service_name"].isin(services)]

In [ ]:
# loop through services_df and download all tiles from each service using the service_url and service_name
for index, row in services_to_download_df.iterrows():
    service_name = row["service_name"]
    print(f"Downloading tiles from service: {row['service_name']}")

    download_and_create_cog(
        service_url=service_url,
        service_name=row["service_name"],
        category="elevation",
        output_path=output_folder / f"{service_name.lower()}.tif",
        bbox_gpkg_path="../data/roi/Verschneidung_NPG_Johnsbachtal_Europaschutzgebiet_BBOX.gpkg",
        raw_data_folder=f"../data/raw/elevation_tiles_gis_stmk/{service_name.lower()}",
        out_nodata_value=-9999
    )

# Create nDSM

DSM and DTM need to have aligned pixels for this to work.

In [ ]:
import rasterio
from rasterio.windows import transform

from src.trainers.utils import read_intersecting_area_of_two_rasters

data_folder = Path("D:/Nextcloud/HabitAlp2.0/Originaldaten/processed/elevation_gis_stmk_photogrammetry_2022_2023")

dsm_array, dtm_array, output_meta = read_intersecting_area_of_two_rasters(
    data_folder / "dsm.tif",
    data_folder / "dtm.tif",
)

output_meta.update(
{
    "compress": "lzw",
    "nodata": -9999,
    "PREDICTOR": 2,
    "tiled":True, 
    "blockxsize":256, 
    "blockysize":256, 
    "BIGTIFF":'YES'
}
)

# Calculate nDSM
ndsm_array = dsm_array - dtm_array

# Save nDSM to new raster file
with rasterio.open(
    data_folder / "ndsm.tif",
    "w",
    **output_meta
) as dst:
    dst.write(ndsm_array, 1)

# Convert Local Elevation Data

In [ ]:
from src.data.processing.processing_utils import create_vrt_from_tiles, create_cog

for type in ["dtm", "dsm", "ndsm"]:
    tiles = list(Path(f"../data/raw/elevation_tiles_waldstmk/{type}").iterdir())
    vrt_path = Path(f"../data/raw/elevation_tiles_waldstmk/{type}.vrt")
    output_path = f"../data/processed/elevation_waldstmk/{type}.tif"

    # Create VRT
    print("Creating VRT...")
    create_vrt_from_tiles(tiles, vrt_path)

    # Convert to COG
    print("Converting to COG...")
    create_cog(vrt_path, output_path, category="elevation", outSRS=32633, out_nodata_value=-9999)

# DEM Processing

In [ ]:
from src.data.processing.processing_utils import dem_processing

dem_processing(
    input_dtm=Path("D:/Nextcloud/HabitAlp2.0/Originaldaten/processed/elevation_gis_stmk_photogrammetry_2022_2023/dtm_2010_2012_aligned.tif"),
    input_dsm=Path("D:/Nextcloud/HabitAlp2.0/Originaldaten/processed/elevation_gis_stmk_photogrammetry_2022_2023/dsm.tif"),
    output_folder=Path("D:/Nextcloud/HabitAlp2.0/Originaldaten/processed/elevation_gis_stmk_photogrammetry_2022_2023"),
)